In [2]:
%matplotlib qt

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from collections import namedtuple

In [4]:
import pandas as pd
import os
from sys import platform
from time import time
from shutil import copy

In [5]:
from pydgilib_extra import DGILibAverages, LOGGER_OBJECT, LOGGER_CSV, LOGGER_PLOT
from experiment.plotting import *

In [6]:
import dgilib_threaded as dgi_t
from atprogram.atprogram import atprogram


In [7]:
import ctypes
try:
    ctypes.windll.shcore.SetProcessDpiAwareness(True)
except:
    pass

Common constants

In [8]:
experiment = "baseline" # or hash
hash_size = 8
mod_accuracy = 5
iterations = 100
duration = 60

hash_size_str = ""
if experiment == "hash":
    hash_size_str = str(hash_size)

File locations for experiments

In [9]:
compile_base_dir = os.path.join(os.path.dirname(os.getcwd()),"KalmanARM")
project_dir = os.path.join(compile_base_dir, "STDIO_Redirect_w_TrustZone")

exprm_base_dir = os.path.join(os.getcwd(), "output")
main_file_from = os.path.join(compile_base_dir, "main_" + experiment + ".c")
main_file_to = os.path.join(project_dir, "main.c")

initial_output = os.path.join(exprm_base_dir, "output_arm.csv")

File locations for plotting

In [10]:
exprm_output_base_folder = os.path.join(os.getcwd(),"output",experiment+hash_size_str)
original_output = os.path.join(exprm_output_base_folder, "original_output.csv")

exprm_output_name = experiment + hash_size_str + "_" + str(iterations) + "iter_output.csv"
exprm_averages_name = experiment + hash_size_str + "_" + str(iterations) + "_averages.csv"
exprm_output = os.path.join(exprm_output_base_folder, exprm_output_name)
exprm_averages = os.path.join(exprm_output_base_folder, exprm_averages_name)

DGILib constants

In [11]:
config = {
    "input_acc_file": os.path.join("input", "input_acc.csv"),
    "input_gyro_file": os.path.join("input", "input_gyro.csv"),
    "output_file": exprm_output
}

dgilib_config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "plot_pins_method": "highlight",
    "gpio_delay_time": 0.0015,
    "log_folder": exprm_output_base_folder
}

# dgilib_config_dict_plot = {
#     "loggers": [LOGGER_OBJECT, LOGGER_CSV, LOGGER_PLOT],
#     "plot_pins_method": "highlight"
# }

**Experiment**

In [12]:
copy(main_file_from, main_file_to)
print("Copied '"+ main_file_from + "' to '" + main_file_to + "'")

Copied 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanARM\main_baseline.c' to 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanARM\STDIO_Redirect_w_TrustZone\main.c'


In [13]:
if True: atprogram(project_dir, verbose=1)

Building file: .././main.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hpl/sercom" -I"../hri" -I"../trustzone" -I"../stdio_redirect" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.91\include"  -O0 -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "main.d" -MT"main.d" -MT"main.o"   -o "main.o" ".././main.c" 
.././main.c: In function 'main':
.././main.c:83:3: warning: statement with no effect [-Wunused-value]
   1+1;
   ^
Finished building: .././main.c
Building target: STDIO_Redirect_w_TrustZone.elf
Invoking: ARM

In [ ]:
data, preprocessed_data = dgi_t.measure(duration, iterations, dgilib_config_dict, input_acc_file=config["input_acc_file"], input_gyro_file=["input_gyro_file"], output_file=config["output_file"], waitForPlot=True)

Save Kalman output

In [ ]:
initial_exprm_output = os.path.join(exprm_base_dir, exprm_output_name)
copy(initial_output, exprm_output)
print("Copied '"+ initial_output + "' to '" + initial_exprm_output + "'")

Save averages data

In [ ]:
avg1 = DGILibAverages(data = data, preprocessed_data = preprocessed_data, average_function="leftpoint", 5)
avg1.calculate_averages_for_pin(1)
avg1.print_averages_for_pin(1)

In [ ]:
avg2 = DGILibAverages(data = data, preprocessed_data = preprocessed_data, average_function="pulse", 5)
avg2.calculate_averages_for_pin(1)
avg2.print_averages_for_pin(1)

**Read averages from saved csv**

In [ ]:
avg_baseline = DGILibAverages()
avg_baseline.read_from_csv(exprm_averages)

print("")
print("-- Baseline --")
avg_baseline.calculate_averages_for_pin(2)
avg_baseline.print_averages_for_pin(2, 5)

**Comparison plot with Intel x86 result**

In [ ]:
f1 = comparison_plot(original_output, baseline_output, 100, "Original", "Baseline")
wait_for_plot(f1, True)